In [1]:
import numpy as np
from keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Flatten, LSTM, Dense, Reshape
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import Model
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [2]:
# Set the input shape
input_shape = (256, 256, 3)  # Handwriting drawings specs

load data

In [3]:
# def images_to_sequence(folder, target_size=(224, 224), sequence_length=224):
#     sequences_list = []
#     labels = []
#     for label in ["HC", "PD"]:
#         label_path = os.path.join(folder, label)
#         for image_name in os.listdir(label_path):
#             image_path = os.path.join(label_path, image_name)
#             img = load_img(image_path, color_mode="grayscale", target_size=target_size)
#             image_array = img_to_array(img)
#             img_array = image_array / 255.0 #normalize
#             sequences = img_array.reshape((sequence_length, -1))
#             sequences_list.append(sequences)
#             labels.append(label)
            
#     sequences = np.array(sequences_list)
#     labels = np.array(labels)
#     return sequences, labels

# drawings_sequences, drawings_labels = images_to_sequence("drawings")
# speech_sequences, speech_labels = images_to_sequence("speech")

# #encode labels to numerical
# label_encoder = LabelEncoder()
# drawings_labels_encoded = label_encoder.fit_transform(drawings_labels)
# speech_labels_encoded = label_encoder.fit_transform(speech_labels)

data augmentation

In [4]:
# Set up data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    './drawings/',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    './drawings/',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='validation')


Found 697 images belonging to 2 classes.


Found 173 images belonging to 2 classes.


In [5]:
def create_handwriting_model(input_shape):
    input_layer = Input(shape=input_shape)

    # Convolutional layers
    conv1 = Conv2D(32, kernel_size=(3,3), activation='relu')(input_layer)
    pool1 = MaxPooling2D(pool_size=(2,2))(conv1)
    norm1 = BatchNormalization()(pool1)

    # You can add more convolutional layers if needed

    # LSTM layers expect sequential input. Since we're dealing with images, we will treat each row of pixels as a sequence.
    # Therefore, we will flatten the 2D image into a series of 1D sequences.
    flattened = Flatten()(norm1)
    reshaped = Reshape((127, 127 * 32))(norm1)  # Change the dimensions according to your model's output
    
    # LSTM layers
    lstm1 = LSTM(128, return_sequences=True)(reshaped)
    lstm2 = LSTM(64)(lstm1)

    # Output layer with a single neuron and sigmoid activation function for binary classification
    output = Dense(1, activation='sigmoid')(lstm2)

    model = Model(inputs=input_layer, outputs=output)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Create the model
model_handwriting = create_handwriting_model(input_shape)
model_handwriting.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 127, 127, 32)      128       
 Normalization)                                                  
                                                                 
 reshape (Reshape)           (None, 127, 4064)         0         
                                                                 
 lstm (LSTM)                 (None, 127, 128)          214681

data processing

In [6]:
history_handwriting = model_handwriting.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

Epoch 1/10
22/22 [==============================] - 24s 1s/step - loss: 0.3869 - accuracy: 0.8551 - val_loss: 0.3546 - val_accuracy: 0.8902
Epoch 2/10
22/22 [==============================] - 18s 800ms/step - loss: 0.3550 - accuracy: 0.8881 - val_loss: 0.3451 - val_accuracy: 0.8902
Epoch 3/10
22/22 [==============================] - 19s 873ms/step - loss: 0.3411 - accuracy: 0.8881 - val_loss: 0.3397 - val_accuracy: 0.8902
Epoch 4/10
22/22 [==============================] - 20s 901ms/step - loss: 0.3394 - accuracy: 0.8881 - val_loss: 0.3412 - val_accuracy: 0.8902
Epoch 5/10
22/22 [==============================] - 19s 883ms/step - loss: 0.3291 - accuracy: 0.8910 - val_loss: 0.3334 - val_accuracy: 0.8902
Epoch 6/10
22/22 [==============================] - 21s 965ms/step - loss: 0.3267 - accuracy: 0.8924 - val_loss: 0.3646 - val_accuracy: 0.8902
Epoch 7/10
22/22 [==============================] - 21s 964ms/step - loss: 0.3249 - accuracy: 0.8881 - val_loss: 0.3432 - val_accuracy: 0.8902
Ep

train model

evaluate, ensemble

In [7]:
model_handwriting.save('handwriting_model.h5')
validation_steps = validation_generator.samples // validation_generator.batch_size
# Predict the classes with the model
validation_generator.reset()  # Ensure the generator is reset before making predictions
predictions = model_handwriting.predict(validation_generator, steps=validation_steps, verbose=1)
# Get the most probable class (since we are using a binary classifier with sigmoid activation, a threshold of 0.5 is used)
predicted_classes = (predictions > 0.5).astype("int32").flatten()  # Flatten to make it a 1D array
# Get the true labels
true_classes = validation_generator.classes[:validation_steps * validation_generator.batch_size]

assert len(predicted_classes) == len(true_classes), "Mismatch between predicted and true labels"
# Generate a classification report
report = classification_report(true_classes, predicted_classes, target_names=validation_generator.class_indices.keys())
print(report)

# Generate a confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print(conf_matrix)

c:\Users\datng\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


5/5 [==============================] - 2s 306ms/step


ValueError: Found input variables with inconsistent numbers of samples: [173, 160]